Note: This a notebook in progress

In [1]:
import pandas as pd
PATH = "filtered.csv"
filtered = pd.read_csv(PATH)

In [2]:
filtered.head(5)

,Unnamed: 0,lemma,id_text,id_line,id_word,label
0,0,3(diš)[]NU,epsd2/admin/u3adm/P124149,3,P124149.3.1,o 1
1,1,udu[sheep]N,epsd2/admin/u3adm/P124149,3,P124149.3.2,o 1
2,2,6(diš)[]NU,epsd2/admin/u3adm/P124149,4,P124149.4.1,o 2
3,3,u[ewe]N,epsd2/admin/u3adm/P124149,4,P124149.4.2,o 2
4,4,2(diš)[]NU,epsd2/admin/u3adm/P124149,5,P124149.5.1,o 3


# Identifying People

Here, I create a dictionary with all the PN as keys. The values are dictionaries that contain profession and role mappings. In the future, I will add professions and roles to the PNs. The goal of this notebook is to finally have a csv with three columns for PN, associated role, and associated profession respectively. 

In [3]:
people = dict()
for row in filtered.itertuples():
    if 'PN' in row.lemma:
        people[(row.lemma, row.id_word)] = dict()

In [20]:
#sanity check
#print(people.keys())

# Generate List of Professions Based on Location

This code is to create a list of professions based on words after PN

In [51]:
#find professions <<INCOMPLETE as of 10.21.19>>
import re
professions = [ "aʾigidu[worker]", 
                "abala[water-drawer]", 
                "abrig[functionary]", 
                "ad.KID[weaver]", 
                "agaʾus[soldier]",
                "arad[slave]",
                "ašgab[leatherworker]",
                "aʾua [musician]", 
                "azlag[fuller]",
                "bahar[potter]",
                "bisaŋdubak[archivist]",
                "damgar[merchant]",
                "dikud[judge]",
                "dubsar[scribe]",
                "en[priest]",
                "erešdiŋir[priestess]",
                "ensik[ruler]",
                "engar[farmer]",
                "enkud[tax-collector]",
                "gabaʾaš[courier]",
                "galamah[singer]",
                "gala[singer]",
                "geme[worker]",
                "gudug[priest]",
                "guzala[official]",
                "idu[doorkeeper]",
                "išib[priest]",
                "kaguruk[supervisor]",
                "kaš[runner]",
                "kiŋgia[messenger]",
                "kinda[barber]", 
                "kinkin[miller]",
                "kiridab[driver]", 
                "kurušda[fattener]", 
                "kuš[official]",
                "lu[person]",
                "lugal[king]",
                "lukur[priestess]",
                "lungak[brewer]",
                "malah[sailor]",
                "muhaldim[cook]",
                "mušendu[bird-catcher]",
                "nagada[herdsman]",
                "nagar[carpenter]",
                "nar[musician]",
                "nargal[musician]", 
                "narsa[musician]", 
                "nin[lady]",
                "nubanda[overseer]",
                "nukirik[horticulturalist]",
                "saŋ.DUN₃[recorder]",
                "saŋŋa[official]",
                "simug[smith]",
                "sipad[shepherd]",
                "sukkal[secretary]",
                "šabra[administrator]",
                "šagia[cup-bearer]",
                "šakkanak[general]",
                # "szej[cook]", this is a verb
                "šidim[builder]",
                "šuʾi[barber]",
                "šukud[fisherman]",
                "tibira[sculptor]",
                "ugula[overseer]",
                "unud[cowherd]",
                # "urin[guard]",
                "UN.IL₂[menial]",
                "ušbar[weaver]",
                "zabardab[official]",
                "zadim[stone-cutter]"]


prev_row = None
for row in filtered.itertuples():
        if prev_row != None and 'PN' in prev_row.lemma and (re.match('^[^\]]*', row.lemma)[0] + "]") in professions:
                people[(prev_row.lemma, prev_row.id_word)]['profession'] = row.lemma
        prev_row = row
        
        
english = set()

for word in words:
    #to get only english use this one
    #english.append((re.search('\[(.*?)\]', str(word)).group()))
    english.add(word[0])
    
print(len(english))

pd.DataFrame(list(english)).to_csv('words_after_PN.csv', index = False)

0


In [52]:
for val in set(english):
    print(val[1:-1])

or we can use a pre-existing list

# Finding Roles

More information on keywords can be found here:
https://github.com/niekveldhuis/Sumerian-network/blob/master/Finding%20Merging%20Roles%20and%20Professions/roles_and_keywords.md 

In [53]:
import re
def match_ki_ta():
    prev_row = None
    i=0
    words = []
    for row in filtered.itertuples():
        if re.match(r".+ta\[\]PN", row.lemma) and 'ki[place]N' in prev_row:
            words.append((row.lemma, row.id_word))
            i+=1
        prev_row = row
    print('Number of roles found:', i)
    return words


def find_keyword_prev(keyword):
    prev_row = None
    i=0
    words = []
    for row in filtered.itertuples():
        if prev_row != None and keyword in row.lemma and 'PN' in prev_row.lemma:
            words.append((prev_row.lemma, prev_row.id_word))
            i+=1
        prev_row = row
    print('Number of roles found:', i)
    return words
    
def find_keyword_next(keyword):
    prev_row = None
    i=0
    words = []
    for row in filtered.itertuples():
        if prev_row != None and keyword in prev_row.lemma and 'PN' in row.lemma:
            words.append((row.lemma, row.id_word))
            i+=1
        prev_row = row
    print('Number of roles found:', i)
    return words

1. Matching rows that have 'ki[place]N' in previous cell and '-ta' in current

In [54]:
#source
for person, id_w in match_ki_ta():
    try:
        people[(person, id_w)]['role'] = 'source'
    except:
        print(person, id_w)

Number of roles found: 9284


2. Matching rows that have 'dab[seize]V/t' in lemmatization

In [55]:
for (person, id_w) in find_keyword_prev('dab[seize]V/t'):
    try:
        people[(person, id_w)]['role'] = 'recipient'
    except:
        print(person, id_w)

Number of roles found: 5102


3. Matching rows that have 'mu.DU[delivery]N' in next cell

In [56]:
# new owner

for (person, id_w) in find_keyword_next('mu.DU[delivery]N'):
    try:
        people[(person, id_w)]['role'] = 'new owner'
    except:
        print((person, id_w))

Number of roles found: 2304


In [57]:
for (person, id_w) in find_keyword_prev('šu[hand]N'):
    try:
        people[(person, id_w)]['role'] = 'recipient'
    except:
        print(person, id_w)

Number of roles found: 2594


5. Matching rows that have 'ŋiri[foot]N' in next cell

In [58]:
for (person, id_w) in find_keyword_next('ŋiri[foot]N'):
    try:
        people[(person, id_w)]['role'] = 'intermediary'
    except:
        print((person, id_w))

Number of roles found: 3154


6. Looks for rows with 'maškim' in previous spot

In [59]:
for (person, id_w) in find_keyword_prev('maškim[administrator]N'):
    try:
        people[(person, id_w)]['role'] = 'representative'
    except:
        print((person, id_w))

Number of roles found: 2072


7. Looking for PNs before and after 'zig[rise]V/i'

In [60]:
for (person, id_w) in find_keyword_next('zig[rise]V/i'):
    try:
        people[(person, id_w)]['role'] = 'source'
    except:
        print((person, id_w))

Number of roles found: 12


In [61]:
for (person, id_w) in find_keyword_prev('zig[rise]V/i'):
    try:
        people[(person, id_w)]['role'] = 'source'
    except:
        print((person, id_w))

Number of roles found: 2383


This is every PN mentioned in the filtered texts

In [62]:
len(people.keys())

46126

In [63]:
people_df = pd.DataFrame(list(people.items()))
people_df.head(4)

,0,1
0,"({d}šul-gi-a-a-mu-ta[]PN, P405489.12.2)",{'role': 'source'}
1,"({d}šul-gi-iri-mu[]PN, P405489.13.1)",{'role': 'recipient'}
2,"({d}utu-ba-ni[]PN, P332036.11.1)",{}
3,"(dingir-ki-ma-at[]PN, P332036.19.3)",{}


# Transfer to CSV

In [64]:
name_word = pd.DataFrame(people_df[0].values.tolist(), index=people_df.index, columns = ['Name', 'id_word'])
final = name_word.join(people_df.drop(people_df.columns[0], axis = 1))

pd.concat([final.drop([1], axis=1), final[1].apply(pd.Series)], axis=1)

,Name,id_word,role,profession
0,{d}šul-gi-a-a-mu-ta[]PN,P405489.12.2,source,NaN
1,{d}šul-gi-iri-mu[]PN,P405489.13.1,recipient,NaN
2,{d}utu-ba-ni[]PN,P332036.11.1,NaN,NaN
3,dingir-ki-ma-at[]PN,P332036.19.3,NaN,NaN
4,di-ku₅-i₃-li₂[]PN,P332036.19.6,NaN,NaN
5,la-gi-ip[]PN,P332036.23.3,NaN,NaN
6,a₂-bi₂-lum[]PN,P332036.24.3,NaN,NaN
7,dam-ku-um[]PN,P332036.25.3,NaN,NaN
8,šu-ma-ma[]PN,P332036.26.3,NaN,nubanda[overseer]N
9,a-hu-we-er[]PN,P332036.47.1,recipient,NaN


In [65]:

final.to_csv('roles_professions.csv')

In [50]:
#TODO: need to deal with case when PN not right next to role
[(row.lemma, row.id_word) for row in filtered.itertuples() if 'P330639.17' in row.id_word]

[('a-hu-ni[]PN', 'P330639.17.1'),
 ('šagia[cup-bearer]N', 'P330639.17.2'),
 ('maškim[administrator]N', 'P330639.17.3')]